# Deploying an HDP cluster with Ambari Blueprints

Below we will make 3 HTTP requests using curl:

1. Upload the Blueprint
2. Create the Cluster
3. Check Cluster creation status

You can copy and paste to run directly from your terminal. Use this as an example for sending from your preferred tool or programming language.

The ambari-api-examples ipython notebook _(in this same directory)_ has these examples and many others using Python.

## Other related API URLs

You can browse these directly in the browser as they are GET requests:

  * List Blueprints: http://hostname:8080/api/v1/blueprints
  * Lists Clusters: http://hostname:8080/api/v1/clusters
  * Lists Hosts: http://hostname:8080/api/v1/hosts
  * Export Cluster configuration as Blueprint: http://hostname:8080/api/v1/clusters/clustername?format=blueprint


_Note: Please do not send passwords in the clear when working on a real cluster._

### 1. Upload the Blueprint

Explanation of the curl command:

    -u admin:password
      * sets the username and password.
      * Alternatively, run `-u admin` to be prompted for the password.
    -H X-Requested-By:seano
      * Ambari requires the user to identify themselves.
      & This can be set to anything you like (instead of 'seano')
    -X POST -d @blueprint_hdp22_multinode-simple.blueprint.json
      * Tells curl to submit the blueprint file

In [6]:
%%bash

## Upload the Blueprint

ambari_server=pregion-sean01.cloud.hortonworks.com ## This is the hostname of the ambari-server.
                       ##   - Set to localhost if running directly on the server.
pass=admin        ## The Ambari admin password

curl -u admin:${pass} \
  -H X-Requested-By:seano \
  -X POST -d @blueprints/hdp-201503_blueprint.json \
  http://${ambari_server}:8080/api/v1/blueprints/hdp-201503

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2429    0     0  100  2429      0   1520  0:00:01  0:00:01 --:--:--  1520


### 2. Create the Cluster

In [7]:
%%bash

## Create the cluster

ambari_server=pregion-sean01.cloud.hortonworks.com
pass=admin

curl -u admin:${pass} \
  -H X-Requested-By:seano \
  -X POST \
  -d @blueprints/hdp-201503_cluster.json \
  http://${ambari_server}:8080/api/v1/clusters/mycluster

{
  "href" : "http://pregion-sean01.cloud.hortonworks.com:8080/api/v1/clusters/mycluster/requests/1",
  "Requests" : {
    "id" : 1,
    "status" : "InProgress"
  }
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1036  100   166  100   870      7     41  0:00:23  0:00:20  0:00:03     0


### 3. Check Cluster creation status

The command above will give you URL to check the Cluster creation status.

Or you can check the status from the Ambari Web UI.

In [ ]:
%%bash

ambari_server=hostname
pass=mypassword

curl -u admin:${pass} http://${ambari_server}:8080/api/v1/clusters/mycluster/requests/1

## Extra: Deploy with ambari-bootstrap

Below uses `ambari-bootstrap` to deploy Ambari Agents & Server. I used this to prepare my cluster for the deployment above.

This can be used on clean RHEL/CentOS 6 systems.

In [ ]:
%%bash

# change this to fit your configuration 
ambari_server=privateIPorHostnameOfTheServer

## the URL to the installer script
bootstrap_url=https://raw.githubusercontent.com/seanorama/ambari-bootstrap/master/ambari-bootstrap.sh
 
## install the ambari-server
pdsh -w ${ambari_server} "curl -sSL ${bootstrap_url} | install_ambari_server=true sh"

## install to all other nodes. See ‘man pdsh’ for the various ways you can specify hosts.
pdsh -w ${stack}0[2-3].cloud.hortonworks.com "curl -sSL ${bootstrap_url} | ambari_server=${ambari_server} sh"